In [1]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score
from torch_geometric.data import Data, DataLoader
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch import linalg as LA

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import sys
DIR='/content/drive/My\ Drive/pan/'
sys.path.append(DIR)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mounted at /content/drive
cuda


In [4]:
def non_ptb_data(index, n, x, y):
    # prepare the non-perturbed data
    edge_index = np.concatenate((index,index[:,[1,0]]),axis=0)
    edge_index = torch.tensor(edge_index,dtype=torch.long)
    x = torch.as_tensor(x,dtype=torch.float)
    y = torch.as_tensor(y,dtype=torch.float)
    data = Data(x=x,edge_index=edge_index.t().contiguous(),num_nodes=n, y=y)
    return data

def ptb_data_train(index, n, x, label):
    # prepare the perturbed data with the edge (s,t) added or removed
    # if label == 1 then remove the edge, and add the edge otherwise
    labels = np.zeros((1,2))
    labels[0,0] = label[2]
    labels[0,1] = label[2]
    if label[2] == 0:
        edge = np.zeros((1,2))
        edge[0,0] = label[0]
        edge[0,1] = label[1]
        edge_index = np.concatenate((index,edge),axis=0)
        data = non_ptb_data(edge_index,n,x,labels)
    else:
        ind = np.where((index == (label[0], label[1])).all(axis=1))
        edge_index = np.delete(index,ind[0],0)
        data = non_ptb_data(edge_index,n,x,labels)
    return data

def ptb_data_test(index, n, x, label):
    # prepare the perturbed data with the edge (s,t) added or removed
    # if label == 1 then remove the edge, and add the edge otherwise
    labels = np.zeros((1,2))
    labels[0,0] = 0
    labels[0,1] = label[2]

    edge = np.zeros((1,2))
    edge[0,0] = label[0]
    edge[0,1] = label[1]
    edge_index = np.concatenate((index,edge),axis=0)
    data = non_ptb_data(edge_index,n,x,labels)
    return data

In [5]:
fname ='/content/drive/My Drive/pan/neural2'
#fname ='/content/drive/My Drive/pan/USAir'
n = 297
batch_size = 100
x = torch.eye(n,dtype=torch.float)
#x = torch.ones((n,1))
train_data = pd.read_csv(fname+'_train_el.csv',header=None)
train_edge_index = np.array(train_data.iloc[:,0:])
non_perturb_train = non_ptb_data(train_edge_index,n,x,np.zeros((1,2))).to(device)





train_labels = pd.read_csv(fname+'_train_labels.csv',header=None)
train_labels = np.array(train_labels.iloc[:,0:])
test_labels = pd.read_csv(fname+'_test_labels.csv',header=None)
test_labels = np.array(test_labels.iloc[:,0:])




train_data_list = [ptb_data_train(train_edge_index,n,x,train_labels[i,0:3]) for i in range(0,len(train_labels))]
train_loader = DataLoader(train_data_list,batch_size=batch_size,shuffle=True)
test_data_list = [ptb_data_test(train_edge_index,n,x,test_labels[i,0:3]) for i in range(0,len(test_labels))]
test_loader = DataLoader(test_data_list,batch_size=int(len(test_data_list)),shuffle=True)

In [11]:


from torch_geometric.nn import MessagePassing

class WRNN(torch.nn.Module):
    def __init__(self,input_size):
        super(WRNN, self).__init__()
        self.conv1 = GCNConv_nonorm(input_size, input_size)
    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        return h

class GCNConv_nonorm(MessagePassing):
    def __init__(self, in_channels, out_channels,diag=False):
        super(GCNConv_nonorm, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.diag=diag
        if self.diag:
            self.diagele=nn.Parameter(torch.ones(in_channels).clone().detach()).to(device)
            self.mat=torch.diag(self.diagele)
        else:
            self.lin = torch.nn.Linear(in_channels, out_channels,bias=False)
            self.lin.weight = nn.Parameter(torch.eye(in_channels).clone().detach())

    def forward(self, x, edge_index):
        if self.diag:
            x= torch.matmul(x,self.mat)
        else:
            x = self.lin(x)
            


        
        norm = torch.ones(edge_index.size(1),1).to(device)
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

class MLP(torch.nn.Module):
    def __init__(self,input_size):
        super(MLP, self).__init__()
        self.linear1 = torch.nn.Linear(input_size,200)
        self.linear2 = torch.nn.Linear(200,100)
        self.linear3 = torch.nn.Linear(100,20)
        self.linear4 = torch.nn.Linear(20,1)
        self.act1= nn.ReLU()
        self.act2= nn.ReLU()
        self.act3= nn.ReLU()

    def forward(self, x):
        out= self.linear1(x)
        out = self.act1(out)
        out= self.linear2(out)
        out = self.act2(out)
        out = self.linear3(out)
        out = self.act3(out)
        out = self.linear4(out)
        out = torch.sigmoid(out)
        return out


class GraphNet(torch.nn.Module):
    def __init__(self, hidden_channels,walk_len=7):
        super(GraphNet, self).__init__()
        #self.lin = torch.nn.Linear(non_perturb_train.num_node_features,hidden_channels)
        self.wrnn = WRNN(hidden_channels)
        self.walk_len = walk_len
    def forward(self, x_p, x_np, y, edge_index_p,edge_index_np):
        #h_p = x_p
        #h_np = x_np
        h_p = self.wrnn(x_p,edge_index_p)
        h_np = self.wrnn(x_np,edge_index_np)
        scale_factor = LA.norm(h_p,dim=0)
        scale_factor = scale_factor[0]
        h_p = h_p/scale_factor
        h_np = h_np/scale_factor
        h_p = self.wrnn(h_p,edge_index_p)
        h_np = self.wrnn(h_np,edge_index_np)
        h_p = h_p/scale_factor
        h_np = h_np/scale_factor
        batch_size = y.size(0)

        p_list = torch.zeros(batch_size,self.walk_len)
        np_list = torch.zeros(1,self.walk_len)
        
        for i in range(0,self.walk_len):
            h_p = self.wrnn(h_p,edge_index_p)
            h_np = self.wrnn(h_np,edge_index_np)
            h_p = h_p/scale_factor
            h_np = h_np/scale_factor
            #h_p = h_p.relu()
            #h_np = h_np.relu()
            val = torch.trace(h_np)
            #np_list[0,i] =  torch.sign(val)*torch.log(torch.abs(val))
            np_list[0,i] = val
            for j in range(batch_size):
                val = torch.trace(h_p[j*n:j*n+n,:])
                #p_list[j,i] = torch.sign(val)*torch.log(torch.abs(val))
                #p_list[j,i] = torch.sign(val)*torch.log(torch.abs(val))
                p_list[j,i] = val
        np_list = np_list.repeat(batch_size,1)
        p_list = p_list- np_list
        p_list = p_list.to(device)
        p_list=p_list*100
        for i in range(0,batch_size):
            p_list[i,:] = torch.mul(p_list[i,:],(y[i,0]-0.5)*2)
        mu = torch.mean(p_list,dim=0,keepdim=False)
        std = torch.std(p_list,dim=0,keepdim=False)
        p_list = (p_list-mu)/std
        #print('P_list,shape',p_list.shape)
        return p_list



In [13]:
IfTrainGraph = True
walk_len=8
Graphmodel = GraphNet(hidden_channels=n,walk_len=walk_len).to(device)
graphout=[]
datalist=[]



for step, data in enumerate(train_loader):
    data = data.to(device)
    datalist.append(data)
    with torch.no_grad():
        out = Graphmodel(data.x, non_perturb_train.x, data.y, data.edge_index, non_perturb_train.edge_index)
        graphout.append(out.clone().detach())
        #graphout.append(torch.randn_like(out))
criterion = torch.nn.BCELoss()
#criterion =torch.nn.MSELoss()
mlpmodel=MLP(walk_len).to(device)

optimizergraph = torch.optim.Adam(Graphmodel.parameters(), lr=0.0001)
#optimizergraph = torch.optim.SGD(Graphmodel.parameters(), lr=-0.001)
optimizermlp = torch.optim.Adam(mlpmodel.parameters(), lr=0.01)


In [14]:

def train(train_loader,IfTrainGraph,IfTrainMLP):
    Graphmodel.train()
    mlpmodel.train()
    loss_eposch=0
    for step, data in enumerate(train_loader):# Iterate in batches over the training dataset.
        data = data.to(device)
        if IfTrainGraph:
            out = Graphmodel(data.x, non_perturb_train.x, data.y, data.edge_index, non_perturb_train.edge_index)
            out = mlpmodel(out)
        else:
            out = mlpmodel(graphout[step])
        target=data.y[:,0]
        out = out.reshape(-1).to(device) # Perform a single forward pass.
        loss = criterion(out, target)  # Compute the loss.
        loss.backward() 
        if IfTrainMLP:
             optimizermlp.zero_grad()
             optimizermlp.step()
        if IfTrainGraph:
             optimizergraph.zero_grad()
             optimizergraph.step()
        if step==8:
            break





        # nloss = -loss.clone()
        # if IfTrainMLP:
        #      optimizermlp.zero_grad()
        #      loss.backward(retain_graph=True) 
        #      optimizermlp.step()
        #      #loss.backward() 
        # if IfTrainGraph:
        #      optimizergraph.zero_grad()
        #      nloss.backward(retain_graph=True)
        #      optimizergraph.step()
        # if step==8:
        #     break
             



        #optimizer.step()  # Update parameters based on gradients.
        #optimizer.zero_grad()  # Clear gradients.
        #print(f"Step:{step},Loss:{loss.item()}")
        loss_eposch=loss_eposch+loss.item()
        #print(loss.item())
        
    return loss_eposch/(step+1.0)

def test(loader):
        mlpmodel.eval()
        Graphmodel.eval()
        with torch.no_grad():
            correct = 0
            for step, data in enumerate(loader):  # Iterate in batches over the training/test dataset.
                #print(f'Num of graphs in the current batch: {data.num_graphs}')
                data = data.to(device)
                out = Graphmodel(data.x, non_perturb_train.x, data.y, data.edge_index, non_perturb_train.edge_index)
                out = mlpmodel(out)
                scores = out.cpu().detach().numpy()
                #print(scores)
                labels = data.y[:,1].cpu().detach().numpy()
                break
        return roc_auc_score(labels, scores)  # Derive ratio of correct predictions.

num_epochs = 10000
for epoch in range(num_epochs):
    # if epoch<10:
    #     traing=False
    # else:
    #     traing=True
    loss_step = train(datalist,True, True)
    if epoch%1==0:
        train_acc = test(datalist)
        test_acc = test(test_loader)
    print('Epoch [{}/{}], Loss: {:.4f}, Train AUC: {:.4f}, Test AUC: {:.4f}'\
                    .format(epoch+1, num_epochs, loss_step, train_acc, test_acc))

RuntimeError: ignored

In [ ]:
mlpmodel.train()
loss_eposch=0
optimizermlp = torch.optim.Adam(mlpmodel.parameters(), lr=0.001)

for i in range(10000):
    for step, data in enumerate(train_loader):# Iterate in batches over the training dataset.
        data = data.to(device)
        print(graphout[step])
        out = mlpmodel(graphout[step])
        target=data.y[:,0]
        out = out.reshape(-1).to(device) # Perform a single forward pass.
        loss = criterion(out, target)  # Compute the loss.
        optimizermlp.zero_grad()
        loss.backward() 
        optimizermlp.step()
        print(target)
        if step==0:
            break

tensor([[-0.2239,  0.0163,  0.2163,  0.2837,  0.3191,  0.3329],
        [-5.4387, -3.7148, -5.1803, -4.4179, -4.8807, -4.6477],
        [-1.7537, -0.7681, -0.9085, -0.6658, -0.6421, -0.5880],
        [ 0.3868,  0.3318,  0.2423,  0.2364,  0.2030,  0.2025],
        [-0.8353,  0.3309,  0.1894,  0.3558,  0.3403,  0.3693],
        [-2.0503, -1.7069, -1.6863, -1.5607, -1.5167, -1.4809],
        [-1.1416, -0.5294, -0.4998, -0.3289, -0.2954, -0.2454],
        [-0.8353, -1.5526, -1.3743, -1.5134, -1.4553, -1.4902],
        [ 0.6924,  0.6038,  0.4976,  0.5151,  0.4888,  0.4973],
        [-0.2232, -0.0596, -0.4026, -0.2983, -0.4082, -0.3749],
        [ 0.6924,  0.8083,  0.6432,  0.6512,  0.6172,  0.6169],
        [ 0.6924,  0.8592,  0.6908,  0.7043,  0.6691,  0.6691],
        [ 0.6924,  0.7741,  0.6349,  0.6417,  0.6060,  0.6056],
        [ 0.0818, -0.8875, -0.1825, -0.4344, -0.2340, -0.2927],
        [ 0.6924,  0.7482,  0.6341,  0.6242,  0.5943,  0.5893],
        [-0.2239,  0.2124,  0.2895,  0.3

ValueError: ignored

In [ ]:
for step, data in enumerate(train_loader):
    data=data.to(device)
    break
print(step)
target=data.y[:,0]
mlpmodel=MLP(walk_len).to(device)
optimizermlp = torch.optim.Adam(mlpmodel.parameters(), lr=0.001)

class MLP(torch.nn.Module):
    def __init__(self,input_size):
        super(MLP, self).__init__()
        self.linear1 = torch.nn.Linear(input_size,2000)
        self.linear2 = torch.nn.Linear(2000,200)
        self.linear3 = torch.nn.Linear(200,1)
        self.act1= nn.ReLU()
        self.act2= nn.ReLU()

    def forward(self, x):
        out= self.linear1(x)
        out = self.act1(out)
        out= self.linear2(out)
        out = self.act2(out)
        out = self.linear3(out)
        out = torch.sigmoid(out)
        return out



0


In [ ]:
out-target

tensor([ 1.0000e+00,  2.0813e-09,  7.6808e-10,  4.3717e-09, -4.1217e-04,
         0.0000e+00,  0.0000e+00,  6.0480e-10,  0.0000e+00,  2.2402e-09,
         0.0000e+00,  2.7822e-06,  0.0000e+00,  2.2246e-07,  1.7201e-08,
         7.8132e-05,  1.5794e-09,  1.0000e+00,  0.0000e+00, -3.4809e-05,
         1.4709e-07, -1.0000e+00,  8.0621e-05,  0.0000e+00,  2.7633e-09,
         0.0000e+00], device='cuda:0', grad_fn=<SubBackward0>)

In [ ]:
for i in range(10000):
    print(graphout[step])
    out = mlpmodel(graphout[step])
    out = out.reshape(-1).to(device)
    loss = criterion(out,target)
    optimizermlp.zero_grad()
    loss.backward() 
    optimizermlp.step()
    print(i,loss.item())
    print(target)


Streaming output truncated to the last 5000 lines.
        [ 0.6924,  0.5696,  0.6018,  0.6053,  0.5932,  0.5957],
        [-0.2239, -0.3500, -0.4444, -0.4591, -0.4858, -0.4891],
        [-0.5296, -0.6913, -0.3232, -0.2962, -0.2056, -0.1795],
        [-0.5296,  0.1181,  0.1745,  0.2946,  0.3197,  0.3482],
        [-0.2239, -0.0346,  0.0860,  0.1622,  0.1673,  0.1877],
        [ 0.3875,  0.0589,  0.3098,  0.2648,  0.2912,  0.2871],
        [ 0.6924,  0.2976,  0.2779,  0.1790,  0.1623,  0.1360],
        [ 0.0818,  0.4336,  0.4471,  0.5304,  0.5340,  0.5539],
        [-0.5296,  0.2976,  0.2994,  0.4089,  0.4101,  0.4294],
        [ 0.6924,  0.7999,  0.5448,  0.5595,  0.4865,  0.4914],
        [ 0.6924,  0.7657,  0.6267,  0.6293,  0.5932,  0.5898],
        [ 0.0818,  0.4595,  0.4273,  0.4751,  0.4743,  0.4845],
        [ 0.0818, -0.6655, -0.2077, -0.3864, -0.2726, -0.3104],
        [ 0.0818, -0.0605,  0.1377,  0.2022,  0.2337,  0.2674],
        [ 0.6924,  0.3318,  0.4252,  0.4030,  0.4157,

KeyboardInterrupt: ignored

In [ ]:
a=[]
for step, data in enumerate(train_loader):
    a.append(data.y[:,0])

In [ ]:
for step, data in enumerate(train_loader):
    print(a[step]-data.y[:,0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.

In [ ]:
GCN_onelay=GCNConv_nonorm(1,1).to(device)

Parameter containing:
tensor([[-0.8094]], requires_grad=True)
haha


In [ ]:
torch.eye_like()

AttributeError: ignored